In [23]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from read_tab_files import TabFileReader

In [24]:
barb_forms = TabFileReader.tab_reader(
    "chl2024_barbacoandata/chl2023_barbacoan_forms.tab"
)

barb_word_list = TabFileReader.get_word_list(barb_forms)
sorted_barb_word_list = sorted(list(set(barb_word_list)))
for word in sorted_barb_word_list[0:20]:
    print(word)

a
a ti
a'san
a'san sili
a'tan
aawa
adyu
afu
aj
ajkesha
ajpa
akin
aman
amana
amta
an
ana
ani
ani srɨ
anku


In [25]:
# convert words to character sequences
tokens = [list(word) for word in barb_word_list]

# flatten the list of chars to fit LabelEncoder
flat_tokens = [item for sublist in tokens for item in sublist]

# encode characters as integers
encoder = LabelEncoder()
encoder.fit(flat_tokens)
encoded_tokens = [encoder.transform(word) for word in tokens]

# specify sequence length (how long each seq should be interpreted as)
sequence_length = 3

X = []
y = []

# go through each word and get every sequence with the seq len (hello > hel, ell, llo if seq_len = 3)
for seq in encoded_tokens:
    for i in range(sequence_length, len(seq)):
        X.append(seq[i - sequence_length : i])
        y.append(seq[i])


# prep data for training
X = np.array(X)
# reshape X to fit the LSTM input [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
y = to_categorical(y)

In [26]:
barb_model = Sequential(
    [
        # memory units (50 is good starting num), seq_len, step size (1)
        LSTM(100, input_shape=(X.shape[1], X.shape[2])),
        Dropout(0.2),
        # num unique chars
        Dense(y.shape[1], activation="softmax"),
    ]
)

barb_model.compile(loss="categorical_crossentropy", optimizer="adam")
barb_model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
64/64 [==============================] - 1s 2ms/step - loss: 3.0386
Epoch 2/100
64/64 [==============================] - 0s 2ms/step - loss: 2.7972
Epoch 3/100
64/64 [==============================] - 0s 2ms/step - loss: 2.7514
Epoch 4/100
64/64 [==============================] - 0s 2ms/step - loss: 2.7311
Epoch 5/100
64/64 [==============================] - 0s 2ms/step - loss: 2.7131
Epoch 6/100
64/64 [==============================] - 0s 2ms/step - loss: 2.6907
Epoch 7/100
64/64 [==============================] - 0s 2ms/step - loss: 2.6766
Epoch 8/100
64/64 [==============================] - 0s 2ms/step - loss: 2.6671
Epoch 9/100
64/64 [==============================] - 0s 2ms/step - loss: 2.6497
Epoch 10/100
64/64 [==============================] - 0s 2ms/step - loss: 2.6317
Epoch 11/100
64/64 [==============================] - 0s 2ms/step - loss: 2.6259
Epoch 12/100
64/64 [==============================] - 0s 2ms/step - loss: 2.6016
Epoch 13/100
64/64 [=================

In [27]:
with open("english_word_list.txt", "r") as file:
    english_word_list = file.read().splitlines()

In [29]:
# convert words to character sequences
eng_tokens = [list(word) for word in english_word_list]

# flatten the list of chars to fit LabelEncoder
eng_flat_tokens = [item for sublist in eng_tokens for item in sublist]

# encode characters as integers
eng_encoder = LabelEncoder()
eng_encoder.fit(eng_flat_tokens)
eng_encoded_tokens = [eng_encoder.transform(word) for word in eng_tokens]

# specify sequence length (how long each seq should be interpreted as)
eng_sequence_length = 3

eng_X = []
eng_y = []

# go through each word and get every sequence with the seq len (hello > hel, ell, llo if seq_len = 3)
for seq in eng_encoded_tokens:
    for i in range(eng_sequence_length, len(seq)):
        eng_X.append(seq[i - eng_sequence_length : i])
        eng_y.append(seq[i])


# prep data for training
eng_X = np.array(eng_X)
# reshape X to fit the LSTM input [samples, time steps, features]
eng_X = np.reshape(eng_X, (eng_X.shape[0], eng_X.shape[1], 1))
eng_y = to_categorical(eng_y)

In [30]:
english_model = Sequential(
    [
        # memory units (50 is good starting num), seq_len, step size (1)
        LSTM(100, input_shape=(eng_X.shape[1], eng_X.shape[2])),
        Dropout(0.2),
        # num unique chars
        Dense(eng_y.shape[1], activation="softmax"),
    ]
)

english_model.compile(loss="categorical_crossentropy", optimizer="adam")
english_model.fit(eng_X, eng_y, epochs=100, verbose=1)

Epoch 1/100
59/59 [==============================] - 1s 2ms/step - loss: 2.9310
Epoch 2/100
59/59 [==============================] - 0s 2ms/step - loss: 2.7879
Epoch 3/100
59/59 [==============================] - 0s 2ms/step - loss: 2.7639
Epoch 4/100
59/59 [==============================] - 0s 2ms/step - loss: 2.7584
Epoch 5/100
59/59 [==============================] - 0s 2ms/step - loss: 2.7475
Epoch 6/100
59/59 [==============================] - 0s 2ms/step - loss: 2.7394
Epoch 7/100
59/59 [==============================] - 0s 2ms/step - loss: 2.7372
Epoch 8/100
59/59 [==============================] - 0s 2ms/step - loss: 2.7294
Epoch 9/100
59/59 [==============================] - 0s 2ms/step - loss: 2.7244
Epoch 10/100
59/59 [==============================] - 0s 2ms/step - loss: 2.7235
Epoch 11/100
59/59 [==============================] - 0s 2ms/step - loss: 2.7228
Epoch 12/100
59/59 [==============================] - 0s 2ms/step - loss: 2.7114
Epoch 13/100
59/59 [=================

In [31]:
def generate_word(model, input_text, extend_length, encoder):
    # pad text if its length is less than seq_len
    if len(input_text) < sequence_length:
        input_text = " " * (sequence_length - len(input_text)) + input_text

    for _ in range(extend_length):
        encoded = encoder.transform(list(input_text[-sequence_length:]))
        encoded = np.reshape(encoded, (1, sequence_length, 1))

        # predict next character
        pred = model.predict(encoded, verbose=0)
        next_char = encoder.inverse_transform([np.argmax(pred)])
        input_text += next_char[0]

    return input_text

In [ ]:
cognates = ["pi", "ku", "mɨ", "lul"]
lengths = list(range(1, 6))

for cognate in cognates:
    print("Cognate:", cognate)
    for length in lengths:
        new_word = generate_word(barb_model, cognate, length, encoder)
        print("  Extra Length:", length)
        print("  Generated Word:", new_word)
    print()

In [ ]:
new_word = generate_word(barb_model, "am", 2, encoder)
print("Generated word:", new_word)

Generated word:  amra


In [37]:
new_word = generate_word(english_model, "the", 2, eng_encoder)
print("Generated word:", new_word)

Generated word: there
